In [37]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import os

In [38]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="code/cred.json"
client = bigquery.Client(project="trkit00")

In [51]:
#parameter
minDate = "2020-02-01"
if 'mindate' in os.environ.keys():
  minDate = os.environ["mindate"]

dataset = "BGN5EL24"


In [40]:
sql = """
SELECT date, COUNT(*) hits
FROM (
SELECT DATE(EventTime) date 
FROM `%s`
WHERE DATE(EventTime) > "%s"
)
GROUP BY date 
ORDER BY date ASC
"""%("trkit00."+dataset+".events", minDate)

In [41]:
df=client.query(sql).to_dataframe()
Output = ""
for i,r in df.iterrows():
    Output += "%s\t%s\n"%(r["date"], r["hits"])

In [47]:
storageClient = storage.Client()
bucket = storageClient.get_bucket('datataskiodemo')
blob = bucket.blob('extractdemoresult.txt')
blob.upload_from_string(Output)